In [66]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import csv
import os
import modeling
import optimization
import tokenization
import tensorflow as tf
import pandas as pd

flags = tf.flags

FLAGS = flags.FLAGS

tf.logging.set_verbosity(tf.logging.INFO)

processors = {
    "cola": "ColaProcessor",
    "mnli": "MnliProcessor",
    "mrpc": "MrpcProcessor",
    "xnli": "XnliProcessor",
    "sim": SimProcessor,
}

In [70]:
class FLAGS():
    do_lower_case = True
    init_checkpoint = "chinese_L-12_H-768_A-12/bert_model.ckpt"
    bert_config_file = "chinese_L-12_H-768_A-12/bert_config.json"
    output_dir = "tmp/sim_model"
    data_dir = "data" 
    task_name = "sim" 
    vocab_file = "chinese_L-12_H-768_A-12/vocab.txt"
    do_train = "true"
    do_eval = "true"
    max_seq_length = 50
    train_batch_size = 32

In [62]:
# # 检查给的参数有没有问题
  # 如果 model是cased model，不能设置--do_lower_case=True`，这样fine-tuning 才能匹配 pre-training model
tokenization.validate_case_matches_checkpoint(FLAGS.do_lower_case,
                                              FLAGS.init_checkpoint)
# # 得到bert_config.__dict__,以字典形式读取config文件, 就能直接用bert_config.xxx的形式调用
bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)

# 新建文件夹
tf.gfile.MakeDirs(FLAGS.output_dir)


In [76]:
task_name = FLAGS.task_name.lower()
processor = processors[task_name]()
label_list = processor.get_labels()
print(label_list)
tokenizer = tokenization.FullTokenizer(
      vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)
print(tokenizer)

['0', '1']


In [42]:
with tf.gfile.GFile(FLAGS.bert_config_file, "r") as reader:
      text = reader.read()


In [50]:
import json
print(json.loads(text))
modeling.BertConfig.from_dict(json.loads(text)).__dict__.keys()

{'attention_probs_dropout_prob': 0.1, 'directionality': 'bidi', 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'intermediate_size': 3072, 'max_position_embeddings': 512, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'pooler_fc_size': 768, 'pooler_num_attention_heads': 12, 'pooler_num_fc_layers': 3, 'pooler_size_per_head': 128, 'pooler_type': 'first_token_transform', 'type_vocab_size': 2, 'vocab_size': 21128}


dict_keys(['vocab_size', 'hidden_size', 'num_hidden_layers', 'num_attention_heads', 'hidden_act', 'intermediate_size', 'hidden_dropout_prob', 'attention_probs_dropout_prob', 'max_position_embeddings', 'type_vocab_size', 'initializer_range', 'directionality', 'pooler_fc_size', 'pooler_num_attention_heads', 'pooler_num_fc_layers', 'pooler_size_per_head', 'pooler_type'])

In [53]:
bert_config.

<bound method BertConfig.to_dict of <modeling.BertConfig object at 0x11f9d7cf8>>

In [ ]:
six.iteritems(json_object)

In [6]:
def validate_case_matches_checkpoint(do_lower_case, init_checkpoint):
  """Checks whether the casing config is consistent with the checkpoint name."""

  # The casing has to be passed in by the user and there is no explicit check
  # as to whether it matches the checkpoint. The casing information probably
  # should have been stored in the bert_config.json file, but it's not, so
  # we have to heuristically detect it to validate.

  if not init_checkpoint:
    return

  m = re.match("^.*?([A-Za-z0-9_-]+)/bert_model.ckpt", init_checkpoint)
  if m is None:
    return

  model_name = m.group(1)

  lower_models = [
      "uncased_L-24_H-1024_A-16", "uncased_L-12_H-768_A-12",
      "multilingual_L-12_H-768_A-12", "chinese_L-12_H-768_A-12"
  ]

  cased_models = [
      "cased_L-12_H-768_A-12", "cased_L-24_H-1024_A-16",
      "multi_cased_L-12_H-768_A-12"
  ]

  is_bad_config = False
  if model_name in lower_models and not do_lower_case:
    is_bad_config = True
    actual_flag = "False"
    case_name = "lowercased"
    opposite_flag = "True"

  if model_name in cased_models and do_lower_case:
    is_bad_config = True
    actual_flag = "True"
    case_name = "cased"
    opposite_flag = "False"

  if is_bad_config:
    raise ValueError(
        "You passed in `--do_lower_case=%s` with `--init_checkpoint=%s`. "
        "However, `%s` seems to be a %s model, so you "
        "should pass in `--do_lower_case=%s` so that the fine-tuning matches "
        "how the model was pre-training. If this error is wrong, please "
        "just comment out this check." % (actual_flag, init_checkpoint,
                                          model_name, case_name, opposite_flag))


In [54]:
class aa():
    pass
dd = aa()
dd.__dict__['g']='a'
dd.g #输出 'a'

In [55]:
dd = aa()

In [57]:
dd.__dict__['g']='a'

In [58]:
dd.g

'a'

In [64]:
class DataProcessor(object):
  """Base class for data converters for sequence classification data sets."""

  def get_train_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the train set."""
    raise NotImplementedError()

  def get_dev_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the dev set."""
    raise NotImplementedError()

  def get_test_examples(self, data_dir):
    """Gets a collection of `InputExample`s for prediction."""
    raise NotImplementedError()

  def get_labels(self):
    """Gets the list of labels for this data set."""
    raise NotImplementedError()

  @classmethod
  def _read_tsv(cls, input_file, quotechar=None):
    """Reads a tab separated value file."""
    with tf.gfile.Open(input_file, "r") as f:
      reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
      lines = []
      for line in reader:
        lines.append(line)
      return lines

class SimProcessor(DataProcessor):
  """Processor for the Sim task"""

  # read csv
  # def get_train_examples(self, data_dir):
  #   file_path = os.path.join(data_dir, 'train.csv')
  #   train_df = pd.read_csv(file_path, encoding='utf-8')
  #   train_data = []
  #   for index, train in enumerate(train_df.values):
  #       guid = 'train-%d' % index
  #       text_a = tokenization.convert_to_unicode(str(train[0]))
  #       # text_b = tokenization.convert_to_unicode(str(train[1]))
  #       label = str(train[1])
  #       train_data.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
  #   return train_data

  # read txt
  def get_train_examples(self, data_dir):
    file_path = os.path.join(data_dir, 'train.txt')
    f = open(file_path, 'r')
    train_data = []
    index = 0
    for line in f.readlines():
        guid = 'train-%d' % index
        line = line.replace("\n", "").split("\t")
        text_a = tokenization.convert_to_unicode(str(line[0]))
        text_b = tokenization.convert_to_unicode(str(line[1]))
        label = str(line[2])
        train_data.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        index += 1
    return train_data

  # csv
  # def get_dev_examples(self, data_dir):
  #   file_path = os.path.join(data_dir, 'dev.csv')
  #   dev_df = pd.read_csv(file_path, encoding='utf-8')
  #   dev_data = []
  #   for index, dev in enumerate(dev_df.values):
  #       guid = 'dev-%d' % index
  #       text_a = tokenization.convert_to_unicode(str(dev[0]))
  #       # text_b = tokenization.convert_to_unicode(str(dev[1]))
  #       label = str(dev[1])
  #       dev_data.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
  #   return dev_data

  def get_dev_examples(self, data_dir):
    file_path = os.path.join(data_dir, 'dev.txt')
    f = open(file_path, 'r')
    dev_data = []
    index = 0
    for line in f.readlines():
        guid = 'dev-%d' % index
        line = line.replace("\n", "").split("\t")
        text_a = tokenization.convert_to_unicode(str(line[0]))
        text_b = tokenization.convert_to_unicode(str(line[1]))
        label = str(line[2])
        dev_data.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        index += 1
    return dev_data

  def get_test_examples(self, data_dir):
    file_path = os.path.join(data_dir, 'test.txt')
    f = open(file_path, 'r')
    test_data = []
    index = 0
    for line in f.readlines():
        guid = 'test-%d' % index
        line = line.replace("\n", "").split("\t")
        text_a = tokenization.convert_to_unicode(str(line[0]))
        text_b = tokenization.convert_to_unicode(str(line[1]))
        label = str(line[2])
        test_data.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        index += 1
    return test_data

  def get_labels(self):
    return ['0', '1']